### Set up functions and routes

In [1]:
from datetime import datetime
from zoneinfo import ZoneInfo


def get_time(timezone: str) -> str:
    """Finds the current time in a specific timezone.

    :param timezone: The timezone to find the current time in, should
        be a valid timezone from the IANA Time Zone Database like
        "America/New_York" or "Europe/London".
    :type timezone: str
    :return: The current time in the specified timezone."""
    now = datetime.now(ZoneInfo(timezone))
    print(f"Invoked `get_time` function with timezone: `{timezone}`")
    return now.strftime("%H:%M")


def get_news(category: str, country: str) -> str:
    """Useful to get the news in a specific country"""
    print(
        f"Invoked: `get_news` function with category: `{category}` "
        f"and country: `{country}`"
    )
    return "Results from dummy news API"

In [2]:
get_time("America/New_York")

Invoked `get_time` function with timezone: `America/New_York`


'05:44'

Now generate a dynamic routing config for each function

In [3]:
from semantic_router import Route

functions = [get_time, get_news]

routes = []

for function in functions:
    route = Route.from_dynamic_route(entity=function)
    routes.append(route)

/Users/jakit/customers/aurelio/semantic-router/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-05 12:44:13 INFO semantic_router.utils.logger Generating dynamic route...
2024-01-05 12:44:16 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_time",
    "utterances": [
        "What's the current time in New York?",
        "Can you tell me the time in London?",
        "What's the current time in Tokyo?",
        "Can you give me the time in Sydney?",
        "What's the current time in Berlin?"
    ]
}
2024-01-05 12:44:16 INFO semantic_router.utils.logger Generating dynamic route...
2024-01-05 12:44:19 INFO semantic_router.utils.logger Generated route config:
{
    "name": "get_news",
    "utterances": [
        "Tell me the latest news from the United States",
   

In [4]:
# You can manually add or remove routes

get_weather_route = Route(
    name="get_weather",
    utterances=[
        "what is the weather in SF",
        "what is the current temperature in London?",
        "tomorrow's weather in Paris?",
    ],
    function_schema=None,
)
routes.append(get_weather_route)

Add routes to the layer config

In [5]:
from semantic_router.layer import LayerConfig

layer_config = LayerConfig(routes=routes)
layer_config.to_dict()

2024-01-05 12:44:23 INFO semantic_router.utils.logger Using default openai encoder: None


{'encoder_type': 'openai',
 'encoder_name': None,
 'routes': [{'name': 'get_time',
   'utterances': ["What's the current time in New York?",
    'Can you tell me the time in London?',
    "What's the current time in Tokyo?",
    'Can you give me the time in Sydney?',
    "What's the current time in Berlin?"],
   'description': None,
   'function_schema': {'name': 'get_time',
    'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.',
    'signature': '(timezone: str) -> str',
    'output': "<class 'str'>"}},
  {'name': 'get_news',
   'utterances': ['Tell me the latest news from the United States',
    "What's happening in India today?",
    'Can you give me the top stories from Japan',
    'Get me the breaking news from the UK',
 

In [6]:
# Get a route by name
layer_config.get("get_time")

Route(name='get_time', utterances=["What's the current time in New York?", 'Can you tell me the time in London?', "What's the current time in Tokyo?", 'Can you give me the time in Sydney?', "What's the current time in Berlin?"], description=None, function_schema={'name': 'get_time', 'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.', 'signature': '(timezone: str) -> str', 'output': "<class 'str'>"})

In [7]:
# Remove a route by name
layer_config.remove("get_weather")
layer_config.to_dict()

2024-01-05 12:44:32 INFO semantic_router.utils.logger Removed route `get_weather`


{'encoder_type': 'openai',
 'encoder_name': None,
 'routes': [{'name': 'get_time',
   'utterances': ["What's the current time in New York?",
    'Can you tell me the time in London?',
    "What's the current time in Tokyo?",
    'Can you give me the time in Sydney?',
    "What's the current time in Berlin?"],
   'description': None,
   'function_schema': {'name': 'get_time',
    'description': 'Finds the current time in a specific timezone.\n\n:param timezone: The timezone to find the current time in, should\n    be a valid timezone from the IANA Time Zone Database like\n    "America/New_York" or "Europe/London".\n:type timezone: str\n:return: The current time in the specified timezone.',
    'signature': '(timezone: str) -> str',
    'output': "<class 'str'>"}},
  {'name': 'get_news',
   'utterances': ['Tell me the latest news from the United States',
    "What's happening in India today?",
    'Can you give me the top stories from Japan',
    'Get me the breaking news from the UK',
 

Save config to a file (.json or .yaml)

In [8]:
layer_config.to_file("output/layer_config.json")

2024-01-05 12:45:12 INFO semantic_router.utils.logger Saving route config to output/layer_config.json


### Define routing layer

Load config from local file

In [9]:
from semantic_router.layer import LayerConfig

layer_config = LayerConfig.from_file("output/layer_config.json")

2024-01-05 12:45:15 INFO semantic_router.utils.logger Loading route config from output/layer_config.json
2024-01-05 12:45:15 INFO semantic_router.utils.logger Using default openai encoder: None


Initialize routing layer

In [11]:
import os
from getpass import getpass
from semantic_router import RouteLayer

# https://dashboard.cohere.com/
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY") or getpass(
    "Enter Cohere API Key: "
)

layer = RouteLayer.from_config(config=layer_config)

2024-01-05 12:45:50 INFO semantic_router.utils.logger Initializing RouteLayer


Do a function call with functions as tool

In [12]:
layer("What is the time in Stockholm?")

2024-01-05 12:45:53 INFO semantic_router.utils.logger Extracting function input...


RouteChoice(name='get_time', function_call={'timezone': 'Europe/Stockholm'})

Define function execution method

In [13]:
from semantic_router.schema import RouteChoice
from semantic_router.utils import llm


def route_and_execute(query, functions, layer):
    route_choice: RouteChoice = layer(query)

    for function in functions:
        if function.__name__ == route_choice.name:
            if route_choice.function_call:
                return function(**route_choice.function_call)

    # If no function is found, use the LLM for general queries
    return llm.llm(query)


queries = [
    "What is the time in Stockholm?",
    "What are the tech news in the US?",
    "The capital of France?",
]

for query in queries:
    print(f"Query: {query}")
    print(route_and_execute(query, functions, layer))

Query: What is the time in Stockholm?


2024-01-05 12:45:58 INFO semantic_router.utils.logger Extracting function input...


Invoked `get_time` function with timezone: `Europe/Stockholm`
11:46
Query: What are the tech news in the US?


2024-01-05 12:46:00 INFO semantic_router.utils.logger Extracting function input...


Invoked: `get_news` function with category: `tech` and country: `US`
Results from dummy news API
Query: The capital of France?
The capital of France is Paris. It's a beautiful city known for its art, culture, and cuisine. Have you ever been there?
